In [1]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__, keras.__version__)

2.6.0 2.6.0


In [2]:
import pandas as pd
import numpy as np
import os
from mat4py import loadmat, savemat

In [3]:
# raw data paths
raw_base = os.path.normpath("D:\Data Division\Raw Data")
eo_adhd = os.path.join(raw_base, 'Eyes-open', 'ADHD')
ec_adhd = os.path.join(raw_base, 'Eyes-closed', 'ADHD')
eo_cont = os.path.join(raw_base, 'Eyes-open', 'Control')
ec_cont = os.path.join(raw_base, 'Eyes-closed', 'Control')

# channel locations path
locs_path = os.path.normpath("D:\Data Division\Standard-10-20-Cap19.locs")

# processed data paths
proc_base = os.path.normpath("D:\MatLab\\toolbox\eeglab2021.0\eeglab2021.0\data_for_processing\EEG_DATA")
# ADJUST
adj_base = os.path.join(proc_base, "Adjust")
adj_eo_adhd = os.path.join(adj_base, 'ADHD_EO')
adj_ec_adhd = os.path.join(adj_base, 'ADHD_EC')
adj_eo_cont = os.path.join(adj_base, 'CONTROL_EO')
adj_ec_cont = os.path.join(adj_base, 'CONTROL_EC')

In [4]:
channels = []
locs = open(locs_path)
for i in locs.read().split('\t'):
    if '\n' in i:
        channels.append(i.split('\n')[0].strip())
channels

['Fp1',
 'Fp2',
 'F7',
 'F3',
 'Fz',
 'F4',
 'F8',
 'T7',
 'C3',
 'Cz',
 'C4',
 'T8',
 'P7',
 'P3',
 'Pz',
 'P4',
 'P8',
 'O1',
 'O2']

In [5]:
print(len(os.listdir(eo_adhd)))
print(len(os.listdir(eo_cont)))

47
50


In [6]:
targets = np.array(([1]*47)+([0]*50))

In [7]:
print(targets.shape)
targets

(97,)


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

In [8]:
data = []
for file in os.listdir(eo_adhd):
    data.append(loadmat(os.path.join(eo_adhd, file))['EO_Data'])

In [9]:
for file in os.listdir(eo_cont):
    data.append(loadmat(os.path.join(eo_cont, file))['EO_Data'])
data = np.array(data)

In [10]:
print(data.shape)
print(len(data[0][0]))
print(data[0].shape)

(97, 46080, 19)
19
(46080, 19)


In [11]:
np.random.seed(88)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

In [13]:
X_train_complete, X_test, y_train_complete, y_test = train_test_split(data, targets)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_complete, y_train_complete, train_size=0.8)

In [14]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_valid.shape, y_valid.shape)

(57, 46080, 19) (57,)
(25, 46080, 19) (25,)
(15, 46080, 19) (15,)


In [15]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
X_valid = scaler.transform(X_valid.reshape(-1, X_valid.shape[-1])).reshape(X_valid.shape)

In [16]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_valid.shape, y_valid.shape)

(57, 46080, 19) (57,)
(25, 46080, 19) (25,)
(15, 46080, 19) (15,)


In [17]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[46080, 19]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 875520)            0         
_________________________________________________________________
dense (Dense)                (None, 300)               262656300 
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               1

In [19]:
model.compile(loss=keras.losses.BinaryCrossentropy(),
              optimizer=keras.optimizers.SGD(learning_rate=0.02),
              metrics=['accuracy'])

In [20]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('eeg_mlp.h5', save_best_only=True)
earlystop_cb = keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, earlystop_cb])